In [1]:
import pandas as pd

In [2]:
# Importing dataset
df=pd.read_csv("dataset/train.csv")

# Checking the NULL values, total rows after removing null values = 1038460

In [3]:
non_null_df=df.dropna()

In [4]:
non_null_df.shape

(1038460, 6)

# Analysing the number of characters in the description, bullet points and title

In [13]:
sum=0
for i in list(non_null_df['BULLET_POINTS']):
    sum=sum+len(i)
print(f"the average character count of BULLET_POINTS is {sum/len(list(non_null_df['BULLET_POINTS']))}")

sum=0
for i in list(non_null_df['TITLE']):
    sum=sum+len(i)
print(f"the average character count of TITLE is {sum/len(list(non_null_df['TITLE']))}")

sum=0
for i in list(non_null_df['DESCRIPTION']):
    sum=sum+len(i)
print(f"the average character count of DESCRIPTION is {sum/len(list(non_null_df['DESCRIPTION']))}")

the average character count of BULLET_POINTS is 545.0705621786107
the average character count of TITLE is 105.9750832964197
the average character count of DESCRIPTION is 642.3917868767213


# Creating an extractive summary for the description and Bullet points

In [9]:
sum_df= non_null_df.copy()
sum_df= sum_df.sample(n=100000)
sum_df['NUMERIC_DATA'] = ''
orig_df= sum_df.copy()
sum_df.shape

(100000, 7)

In [7]:
import yake
import re
# kw_extractor = yake.KeywordExtractor(top=10, stopwords=None)

In [55]:
#testing

# Define the input text containing numeric values
input_text = "This product has a length of 10.2 cm and a width of 5.5 cm. It weighs 200 grams."

# Preprocess the input text by replacing all numeric values with a placeholder string
preprocessed_text = re.sub(r'\d+(\.\d+)?', 'NUMERIC_VALUE', input_text)

# Initialize the YAKE keyword extractor
# w_extractor = yake.KeywordExtractor(top=10, stopwords=None)
kw_extractor = yake.KeywordExtractor()

# Extract keywords from the preprocessed text
keywords = kw_extractor.extract_keywords(preprocessed_text)

# Replace the placeholder string with the original numeric values in the extracted keywords
for keyword,v in keywords:
    print(keyword)
    # keyword['kw'] = re.sub(r'NUMERIC_VALUE', keyword['kw'], input_text)

# Print the extracted keywords
print(keywords)

NUMERIC
length of NUMERIC
width of NUMERIC
weighs NUMERIC
product
length
width
grams
weighs
[('NUMERIC', 0.03827393454912459), ('length of NUMERIC', 0.04333916175368838), ('width of NUMERIC', 0.04333916175368838), ('weighs NUMERIC', 0.05838988829590717), ('product', 0.2102877972083829), ('length', 0.2102877972083829), ('width', 0.2102877972083829), ('grams', 0.37181436490042297), ('weighs', 0.48047146236409)]


In [50]:
# TESTING
column_name = "BULLET_POINTS"
text = list(sum_df['BULLET_POINTS'])[2]
print(text)
keywords = kw_extractor.extract_keywords(text)
print(keywords)



[The Box contains 3 types safety eyes, 50pcs 8mm safety eyes, 50pcs 10mm safety eyes, 40pcs 12mm safety eyes, each size h 3 colors,The Box contains 3 types safety eyes, 50pcs 8mm safety eyes, 50pcs 10mm safety eyes, 40pcs 12mm safety eyes, each size h 3 colors,Whers can prevent the eyes out of the dolls,Suitable for many kinds of puppets, like doll, Teddy Bear, plush animals, etc,Good-quality;very convenient to use, just enjoy yourselves with friends and children,Great for hand making, very convenient to use]
[('types safety eyes', 0.00030770990168920294), ('safety eyes', 0.0010680589118758981), ('Teddy Bear', 0.002187139801792932), ('types safety', 0.0024695724484394615), ('colors,The Box', 0.004353760209951563), ('plush animals', 0.007189015193185823), ('colors,Whers can prevent', 0.00855869088097288), ('kinds of puppets', 0.00855869088097288), ('hand making', 0.00855869088097288), ('safety', 0.008638020951789206)]


In [58]:
# testing

# text = "The price of the product is $25.50 and it weighs 1.5 kilograms."
text = "The Box contains 3 types safety eyes, 50pcs 8mm safety eyes, 50pcs 10mm safety eyes, 40pcs 12mm safety eyes, each size h 3 colors,The Box contains 3 types safety eyes, 50pcs 8mm safety eyes, 50pcs 10mm safety eyes, 40pcs 12mm safety eyes, each size h 3 colors,Whers can prevent the eyes out of the dolls,Suitable for many kinds of puppets, like doll, Teddy Bear, plush animals, etc,Good-quality;very convenient to use, just enjoy yourselves with friends and children,Great for hand making, very convenient to use"

# Create a YAKE keyword extractor
kw_extractor = yake.KeywordExtractor(lan="en", n=1, dedupLim=0.9, top=20)

# Extract keywords and numeric values from the text
keywords = kw_extractor.extract_keywords(text)

# Filter out numeric values
keywords = [keyword[0] for keyword in keywords if not keyword[0].replace(".", "").isnumeric()]

# Extract numeric values
numeric_values = [word for word in text.split() if word.replace(".", "").isnumeric()]

# Add numeric values to keywords
keywords += numeric_values

# Print the keywords
print(keywords)

['safety', 'eyes', 'Box', 'types', 'size', 'convenient', 'Teddy', 'Bear', 'colors,The', 'colors,Whers', 'puppets', 'doll', 'plush', 'animals', 'making', 'prevent', 'dolls,Suitable', 'kinds', 'enjoy', 'friends', '3', '3', '3', '3']


In [50]:
def getKeywords(text):

    # Create a YAKE keyword extractor
    kw_extractor = yake.KeywordExtractor(lan="en", n=1, dedupLim=0.9, top=20)

    # Extract numeric values and replace with "NUMERIC"
    numeric_values = []
    text_tokens = []
    for token in text.split():
        if token.replace(".", "").isnumeric():
            numeric_values.append(token)
            text_tokens.append("NUMERIC")
        else:
            text_tokens.append(token)

    # Convert text tokens back to a string
    text = " ".join(text_tokens)

    # Extract keywords from the modified text
    keywords = kw_extractor.extract_keywords(text)

    # Replace "NUMERIC" with the original numeric values
    for i, keyword in enumerate(keywords):
        if keyword[0] == "NUMERIC":
            keywords[i] = (numeric_values.pop(0), keyword[1])

    opstr=""
    for i in keywords:
        opstr=opstr+i[0]+","
    
    # return the keywords
    return opstr



In [87]:
# testing 

# column_name = "DESCRIPTION"
column_name = "BULLET_POINTS"
# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text = row[column_name]
    print(text+"\n")

Reversible,Wipe,Clean,Water,Proof,Wet,Marks,Dining,table,Ironing,Scratches,Pieces,Refrigerator,Kitchen,Place,Placemat,mat,Multi-Purpose,also.,Set,Size,drawer,

Stylish,removed,Custom,Secure,functions,Durable,Lightweight,Impact,resistant,flexible,case.,fit,easily,applied,Design,Access,ports,plastic,

safe,microwave,oven,Multicolour,Material,free,Highly,Content,Toxic,BPA,Environment,Reusable,Odourless,Piece,Dishwasher,durable,lead,freezer,safe,Color,Ceramic,Package,friendly,

Comfort,Safe,Bezel,Attachment,Unique,Original,Put,Scratch,Double,Styling.,Secure,Design,Designed,Styles,Add,Strong,Styling,sapphire,crystal,Ringke,

Material,hair,headband,tooth,design,Creation,8,clips,made,good,teeth,number,shape,suitable,occasions,resin,flexibility,harm,girls,women,New,

Ports,Camera,Compatible,Vivo,Endured,Sides.,Buttons,Speakers,Methods,Protection,Phone,Case,Precision,Cutouts,Microphone,Product,Vary,Slightly,Colour,Display,

UNIQUE,Room,MAKE,FLOOR,Shaggy,Status,Living,carpet,carpets,Bedroom,hall

In [51]:
# Set the column name
column_name = "DESCRIPTION"

# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text = row[column_name]
    
    # Use yake to extract 20 keywords 
    kw=getKeywords(text)

    # Replace the original text with the summary in the same column
    sum_df.at[index, column_name] = kw

In [9]:
# Set the column name
column_name = "BULLET_POINTS"

# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text = row[column_name]
    
    # Use yake to extract 20 keywords 
    kw=getKeywords(text)

    # Replace the original text with the summary in the same column
    sum_df.at[index, column_name] = kw

In [65]:
sum=0
for i in list(sum_df['BULLET_POINTS']):
    sum=sum+len(i)
print(f"the average character count of BULLET_POINTS is {sum/len(list(sum_df['BULLET_POINTS']))}")

the average character count of BULLET_POINTS is 186.0


In [8]:
sum=0
for i in list(sum_df['DESCRIPTION']):
    sum=sum+len(i)
print(f"the average character count of DESCRIPTION is {sum/len(list(sum_df['DESCRIPTION']))}")

the average character count of DESCRIPTION is 128.85341


In [66]:
print(list(sum_df['BULLET_POINTS'])[2])

TABLE DECOR TABLE DECOR TABLE table DECOR RUNNER DECOR TABLE DECOR RUNNER DECOR RUNNER table RUNNER TABLE DECOR TABLE DECOR HIGH DECOR HIGH QUALITY TABLE TABLE DECOR QUALITY TABLE DECOR 


In [10]:
# saving the dataframe
sum_df.to_csv('dataset/pre-proccessed.csv')

# Checking the dataframe for empty string


In [11]:
counter=0
for i in list(sum_df["BULLET_POINTS"]):
    if(len(i)==0):
        counter=counter+1
print(counter)

247


# Adding REGEX to extract length

In [24]:
#testing
import re

description = 'MUKHTAR CARPETS Handmade Tuffted Floral Carpet for Living Room Bedroom Drawing Room and Hall Size 5 x 8 [ 150 x 240 cm ]'


length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in|inch|ft|foot|feet|yd|yard|')?\b"


lengths = re.findall(length_pattern, description, re.IGNORECASE)

print(lengths)

['5 ', '8', '150 ', '240 cm']


In [39]:
# testing
import re

# description = "This product has dimensions of 3/4\" x 14\"-20\" and weighs 1.5 kg. The size is 5*3*2 cm and the weight is 0.25 lbs."
description = 'MUKHTAR CARPETS Handmade Tuffted Floral Carpet for Living Room Bedroom Drawing Room and Hall Size 5 x 8 [ 150 x 240 cm ]'


# length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in|inch|ft|foot|feet|yd|yard|')?\b"
length_pattern = r'(\d+(?:\.\d+)?(?:/\d+)?(?:\s*\*\s*\d+(?:\.\d+)?(?:/\d+)?){0,2})(?:\s*(?:feet|ft|meters|m|centimeters|cm|inches|in|"))?'

lengths = re.findall(length_pattern, description, re.IGNORECASE)

print(lengths)

['5', '8', '150', '240']


In [26]:
#testing
import re

description = "This product has dimensions of 3/4\" x 14\"-20\" and weighs 1.5 kg. The size is 5*3*2 cm and the weight is 0.25 lbs."

length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"

lengths = re.findall(length_pattern, description, re.IGNORECASE)

combined_lengths = []
current_length = ""
for length in lengths:
    if "*" in length:
        current_length += length
    else:
        if current_length:
            combined_lengths.append(current_length)
            current_length = ""
        combined_lengths.append(length)
if current_length:
    combined_lengths.append(current_length)

final_lengths = []
for length in combined_lengths:
    # check for fractions
    if "/" in length:
        fraction_parts = length.split("/")
        num = int(fraction_parts[0])
        denom = int(fraction_parts[1])
        length = str(num/denom)
    # add unit symbols to output
    if "inch" in length:
        length = length.replace("inch", "in")
    elif "inches" in length:
        length = length.replace("inches", "in")
    elif "foot" in length:
        length = length.replace("foot", "ft")
    elif "feet" in length:
        length = length.replace("feet", "ft")
    final_lengths.append(length)

print(final_lengths)

['3', '4', '14', '20', '1.5 ', '5*3*2 cm', '0.25 ']


In [46]:
import re

def numeric_data_extractor(description):
    # description = "Made,Hard,Polycarbonate,Plastic,3,Case,cover,sides,back,mobile,printed,bumps,protects,scratches,premier,"

    length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"
    # length_pattern = r"\b(?:\d+(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"
    # length_pattern = r'(\d+(?:\.\d+)?(?:/\d+)?(?:\s*\*\s*\d+(?:\.\d+)?(?:/\d+)?){0,2})(?:\s*(?:feet|ft|meters|m|centimeters|cm|inches|in|"))?'

    lengths = re.findall(length_pattern, description, re.IGNORECASE)

    

    opstr=""
    for i in lengths:
        opstr=opstr+i+","
    
    return opstr

In [44]:
    import re
    description = "Made,Hard,Polycarbonate,Plastic,3,Case,cover,sides,back,mobile,printed,bumps,protects,scratches,premier,"

    # length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"
    # length_pattern = r"\b(?:\d+(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"
    # length_pattern = r'(\d+(?:\.\d+)?(?:/\d+)?(?:\s*\*\s*\d+(?:\.\d+)?(?:/\d+)?){0,2})(?:\s*(?:feet|ft|meters|m|centimeters|cm|inches|in|"))?'
    length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in|inch|ft|foot|feet|yd|yard|')?\b"

    lengths = re.findall(length_pattern, description, re.IGNORECASE)

    final_lengths = []
    for length in lengths:
        # check for fractions
        if "/" in length:
            fraction_parts = length.split("/")
            num = float(fraction_parts[0])
            denom = float(fraction_parts[1])
            if denom!=0:
                length = str(num/denom) + '"'
            else:
                length=0
        # add unit symbols to output
        if "inch" in length:
            length = length.replace("inch", "in")
        elif "inches" in length:
            length = length.replace("inches", "in")
        elif "foot" in length:
            length = length.replace("foot", "ft")
        elif "feet" in length:
            length = length.replace("feet", "ft")
        final_lengths.append(length)

    opstr=""
    for i in final_lengths:
        opstr=opstr+i+","
    
    print(opstr) 

3,


In [ ]:
import spacy
import numpy as np
import pandas as pd

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract quantities, prices, and product labels
def extract_ner(data):
    quantities = []
    prices = []
    products = []
    for text in data:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in {"QUANTITY", "MONEY", "PRODUCT"}:  # Use set for faster membership testing
                if ent.label_ == "QUANTITY":  # Check if the entity is a quantity
                    quantities.append(ent.text)
                elif ent.label_ == "MONEY":  # Check if the entity is a monetary value
                    prices.append(ent.text)
                else:  # Entity must be a product
                    products.append(ent.text)
    return quantities, prices, products

# Example DataFrame
# Concatenate the text columns into a single Series
text_data = pd.concat([train_dropped["TITLE"], train_dropped["BULLET_POINTS"],train_dropped["DESCRIPTION"]])

# Extract quantities, prices, and product labels
quantities, prices, products = extract_ner(text_data)

# Convert the lists to NumPy arrays
quantities = np.array(quantities)
prices = np.array(prices)
products = np.array(products)

# Print the extracted quantities, prices, and product labels
print("Quantities:", quantities)
print("Prices:", prices)
print("Products:", products)

In [47]:
# Set the column name
column_name_1 = "TITLE"
column_name_2 = "BULLET_POINTS"
column_name_3 = "DESCRIPTION"

# Loop through each row in the dataframe
for index, row in orig_df.iterrows():
    # Extract the text to be summarized from the column
    text_1 = row[column_name_1]
    text_2 = row[column_name_2]
    text_3 = row[column_name_3]

    inpstr= text_1+" "+text_2+" "+text_3
    
    # Use get numeric data
    ndata=numeric_data_extractor(inpstr)

    # Replace the original text with the summary in the same column
    sum_df.at[index, 'NUMERIC_DATA'] = ndata

In [48]:
sum_df['NUMERIC_DATA']

1740317                                                     
1511738    2 ,2 ,50cm,50cm,5cm,5,5 ,50cm,50cm,5cm,13.45 ,...
294232                                                      
2032819                                           2 Inch,2 ,
1042207                              380 ,2 ,8 ,1 ,2 ,7200 ,
                                 ...                        
132663                                             100,5,6 ,
1678270         10,13 inch,13,17 inch,17,21 inch,21,26 inch,
516397                                        24 ,1,7 ,1,3 ,
1263496                                                     
1563684                                               2 ,2 ,
Name: NUMERIC_DATA, Length: 100000, dtype: object